
## **Sistema de Detección de Interferencias y Deterioro de SNR**

Este informe final integra la metodología completa, desde la generación del dataset de entrenamiento en el laboratorio hasta el proceso de inferencia en la BeagleBone Black (BBB), con un enfoque en la **precisión experimental** y el **control de la adquisición**.

### **1\. Parámetros del Sistema y Arquitectura**

| Parámetro | Valor | Detalle de la Operación |
| :---- | :---- | :---- |
| **Muestras por Ciclo ($N_{total}$)** | ${2,048,000 { muestras IQ}}$ | Cantidad fija de datos brutos por cada ciclo de ${1 { segundo}}$. |
| **Latencia Aceptada** | Mayor a $1 { segundo}$ | La implementación en Python y el volumen de datos imponen la latencia. |
| **Canal FSK** | $50 { kHz}$ de ancho; tonos en $4900 { Hz}$ y $2450 { Hz}$. |  |

---

### **2\. Protocolo Minucioso de Laboratorio para la Generación del Dataset**

El objetivo es manipular controladamente las condiciones de RF para obtener un rango representativo y preciso de valores para las 10 *features* y las 4 clases de etiquetas.

#### **A. Setup Inicial, Calibración y Captura de Referencia (Clase 0/0)**

1. **Conexión Calibrada:** Combinar la salida del **VSG-1 (FSK)** y el **VSG-2 (Interferencia)** en un **Combinador de RF**. Conectar la salida del combinador al **SDR RTL-SDR** (asegurando una línea de ${50 { Ohm}}$).  
2. **Calibración de Potencia ($P_{{ref}}$):** Establecer la potencia del VSG-1 a un nivel alto (ej., ${+10 { dBm}}$). Medir la potencia final que llega al SDR con un **Medidor de Potencia RF**. Este será el valor de referencia **$P_{{ref}}$** que define la condición **Normal**.  
3. **VSG-2 (Interferencia):** Apagado o con salida a $-100 { dBm}$ (piso de ruido).  
4. **SDR y Disparo (Trigger):** $F_c = 137.000 { MHz}$, $F_s = 2.048 { MHz}$. Usar un **Trigger de Nivel** en el software de captura del SDR para asegurar la consistencia de la captura de ${1 { segundo}}$.  
5. **Captura y Etiquetado:** Capturar ${1 { segundo}}$ de IQ. **Etiquetado:** ${0/0}$. Repetir 500-1000 veces.

#### **B. Protocolo de Captura Detallado por Feature**

| Feature Clave (Muestra) | Procedimiento de Laboratorio (Control Preciso) | Rango de Captura | Etiqueta Base (Ej.) |
| :---- | :---- | :---- | :---- |
| **$P_{Ruido, 0}$** | **VSG-2 (AWGN):** Inyectar **AWGN** con ancho de banda limitado (${\\leq 50 { kHz}}$). | Aumentar la potencia del AWGN inyectado en pasos de ${3 { dB}}$ (ej., de $-10 { dBm}$ a $+5 { dBm}$). | **${0/1}$** (Bajo SNR) |
| **Kurtosis y PAPR** | **VSG-2 (Pulsos):** Generar pulsos de **forma rectangular** de alta potencia (${+15 { dBm}}$) y bajo ciclo de trabajo (ej., $0.5 { ms}$ ON / $99.5 { ms}$ OFF). | Variar el ciclo de trabajo (DC) y la potencia de los pulsos para generar picos de diferentes magnitudes. | **${1/0}$** (Interferencia Impulsiva) |
| **$P_{Adj_Ch}$** | **VSG-2 (Tono):** Inyectar un tono simple a $137.0 { MHz} + {100 { kHz}}$ (Clasificado como adyacente). | Variar la potencia del tono de $-5 { dBm}$ a $+10 { dBm}$. | **${1/0}$** (Interferencia Canal Vecino) |
| **P1, P2 y Ratio P1/P2** | **VSG-2 (Tono Fijo):** Inyectar un tono simple a la frecuencia de ${137.0 { MHz}} + {4.9 { kHz}}$ (exactamente en el tono P1). | Variar la potencia de inyección para crear un **desequilibrio conocido** en la relación P1/P2. | **${1/0}$** (Interferencia de Tono) |
| **OBW** | **VSG-2:** Inyectar ruido blanco de ${200 { kHz}}$ de ancho de banda (más ancho que el canal FSK). | Variar la potencia de la señal de banda ancha inyectada. | **${1/0}$** (Interferencia de Banda Ancha) |
| **Sesgo** | **VSG-2:** Inyectar una señal conocida por tener un **espectro asimétrico** (ej., modulación SSB) o una portadora fuerte fuera de centro que desbalancee la amplitud de la onda. | Variar la potencia de la señal asimétrica. | **${1/0}$** (Interferencia Asimétrica) |
| **Ambos (1/1)** | Combinar la reducción de ${15 { dB}}$ de potencia FSK (Bajo SNR) con la inyección de ${5 { dBm}}$ de pulsos (Interferencia Impulsiva). | Repetir el proceso con diferentes combinaciones de ruido y picos. | **${1/1}$** (Ambos) |

#### **C. Obtención de Features en Python (Procesamiento Offline)**

El script Python procesa cada archivo IQ para calcular las 10 *features* y generar la fila del dataset:

| Feature (Columna) | Obtención Práctica en Python |
| :---- | :---- |
| **Potencia Total (RMS) y PAPR** | ${P_RMS} = {np.mean}({np.abs}(x)^2)$; ${PAPR} = {np.max}({np.abs}(x)^2) / {P_RMS}$ |
| **Kurtosis y Sesgo** | ${scipy.stats.kurtosis}({np.abs}(x), {fisher=False})$; ${scipy.stats.skew}({np.abs}(x))$ |
| **Espectrales** | Integración numérica del ${PSD}$ (Welch) para obtener ${P1}, {P2}, {P}_{{Ruido}, 0}, {P}_{{Adj_Ch}}, {OBW}$. |

---

### **3\. Estructura Única del Dataset y Requisitos**

La tabla de entrenamiento final (CSV/Excel) contiene la información para el Random Forest.

#### **A. Estructura de Filas y Columnas**

| Feature 1 | Feature 2 | Feature 3 | ... | Feature 10 | Label 1 (Salida) | Label 2 (Salida) |
| :---- | :---- | :---- | :---- | :---- | :---- | :---- |
| P_RMS | PAPR | Kurtosis | $\\dots$ | OBW | **Label_Interf** | **Label_SNR** |
| ${1.55}$ | ${5.2}$ | ${8.9}$ | $\\dots$ | ${49.8{k}}$ | ${1}$ | ${0}$ |
| $\\dots$ | $\\dots$ | $\\dots$ | $\\dots$ | $\\dots$ | $\\dots$ | $\\dots$ |

#### **B. Requisito Mínimo de Filas**

El dataset debe contener un **mínimo de ${2,000 { filas}}$** para garantizar la robustez del modelo.

---

### **4\. Proceso de Inferencia y Decisión en el BeagleBone Black (BBB)**

El modelo **Random Forest** serializado se carga en el BBB. La operación se controla mediante la página web servida por Ethernet.

#### **A. Disparo de la Captura a Través de Ethernet**

La adquisición de ${1 { segundo}}$ de muestras es costosa en procesamiento, por lo que el usuario debe iniciar la toma de datos.

1. **Tecnología del Servidor:** La BBB debe ejecutar un micro-framework web en Python, como **Flask**, para servir la página por el puerto Ethernet.  
2. **Elemento de Control:** La página web (HTML/JavaScript) debe contener un **botón** claramente etiquetado (ej., **"ANALIZAR 1 SEGUNDO"** o **"DISPARAR CAPTURA"**).  
3. **Disparo de la Tarea:** Cuando el usuario presiona el botón, el código JavaScript envía una **solicitud HTTP POST** (o GET) a una ruta específica del servidor Flask (ej., /api/capturar).  
4. **Ejecución de la Captura:** La función Python asociada en Flask recibe la solicitud y ejecuta la secuencia:  
   * **Llamada a SDR:** Inicializa el driver del SDR (rtl-sdr o pyrtlsdr).  
   * **Captura:** Adquiere los ${2,048,000}$ muestras IQ.  
   * **Procesamiento:** Inicia el cálculo de las 10 *features* (etapa de latencia).  
   * **Inferencia:** Ejecuta la predicción con el modelo Random Forest.  
5. **Respuesta:** Una vez que la inferencia ha terminado (después de $\>1$ segundo), el servidor Flask envía la decisión final y los datos de las *features* de vuelta a la página web para su visualización.

#### **B. Mecanismo de Decisión del BBB**

La decisión se toma en un único paso de predicción por cada segundo de datos capturados.

| Paso | Acción en el BBB | Resultado de la Predicción |
| :---- | :---- | :---- |
| **1\. Adquisición** | Capturar ${2,048,000 { muestras IQ}}$ del SDR. | Vector $x$. |
| **2\. Extracción de Features** | El script Python calcula las **10 *features***. | Vector ${F}_{{in}} = [f_1, f_2, \\dots, f_{10}]$. |
| **3\. Inferencia** | El vector ${F}_{{in}}$ se introduce en el modelo Random Forest. | Vector de Predicción ${P} = [p_{{interf}}, p_{{snr}}]$. |
| **4\. Decisión Final** | El BBB sirve el resultado: ${p_{{interf}}}$ (Hay Interferencias: SÍ/NO) y ${p_{{snr}}}$ (SNR Empeora: SÍ/NO). |  |

El Random Forest actúa como un sistema de diagnóstico dual, proporcionando una evaluación completa de la salud del canal cada vez que el usuario lo solicita.